In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
"Teste 🚗"

'Teste 🚗'

In [3]:
import os

os.chdir("..")
os.chdir("..")

In [4]:
# import logging
# import numpy as np
import pandas as pd
# from datetime import datetime
# from sklearn.preprocessing import LabelEncoder
import sqlite3
from pathlib import Path
# from modelagem.feature_eng.match_analysis import get_storage_ranks, create_main_cols

# from database.create_connection import create_connection
from etl.infra.database import create_connection
# from modelagem.utils.preprocessing.engine import base_pre_processing
from modelagem.utils.logs import logger
from modelagem.train import model_trainer

# Ler 500 colunas no pandas 
pd.set_option('display.max_columns', 500)

In [5]:
# Definindo diretórios base
BASE_DIR = os.path.dirname(Path("__file__").resolve().parent)
FT_DIR = os.path.join(BASE_DIR, "pred_soccer", "database", 'features')
MODEL_DIR = os.path.join(os.path.dirname(BASE_DIR), 'database')
LOG_DIR = os.path.join(os.path.dirname(BASE_DIR), 'logs')

In [6]:
# df_ft = pd.read_csv(os.path.join(FT_DIR, "ft_df.csv"))

In [7]:

""" 
    is_weekend:bool = Sábado = 5, Domingo = 6
    month:int = Mês do jogo
    days_since_last_match_home = Dias desde o último jogo do time da casa
    match_importance:int = 0 ou 1 ((df['month'] >= 10).astype(int))
    goal_avg_diff:int = df['home_team_goal_avg_last_5'] - df['away_team_goal_avg_last_5']
    win_rate_diff:int = df['home_team_last_5_win_rate'] - df['away_team_last_5_win_rate']
    ranking_diff:int = df['home_team_ranking'] - df['away_team_ranking']
    match_day_of_week_encoder:int = label_encoder.transform(df['match_day_of_week'])
    season_phase_encoder:int = label_encoder.transform(df['season_phase'])


    head_to_head_win_home_team
    head_to_head_draws
    head_to_head_goal_diff
    
"""

" \n    is_weekend:bool = Sábado = 5, Domingo = 6\n    month:int = Mês do jogo\n    days_since_last_match_home = Dias desde o último jogo do time da casa\n    match_importance:int = 0 ou 1 ((df['month'] >= 10).astype(int))\n    goal_avg_diff:int = df['home_team_goal_avg_last_5'] - df['away_team_goal_avg_last_5']\n    win_rate_diff:int = df['home_team_last_5_win_rate'] - df['away_team_last_5_win_rate']\n    ranking_diff:int = df['home_team_ranking'] - df['away_team_ranking']\n    match_day_of_week_encoder:int = label_encoder.transform(df['match_day_of_week'])\n    season_phase_encoder:int = label_encoder.transform(df['season_phase'])\n\n\n    head_to_head_win_home_team\n    head_to_head_draws\n    head_to_head_goal_diff\n    \n"

In [8]:
HOME_TEAM = "Palmeiras"
AWAY_TEAM = "Corinthians"

model_trainer.predict_per_time(home_team=HOME_TEAM, away_team=AWAY_TEAM, list_seasons=[2023, 2022])

2025-04-15 15:52:10,777 | main | INFO | model_trainer | load_model | Modelo carregado com sucesso de: database\models\logistic_regression_model.pkl


(np.int64(2), array([0.06652832, 0.30820007, 0.62527162]))

In [9]:
# ['home_team_last_5_win_rate', 'away_team_last_5_win_rate',
#        'home_team_goal_avg_last_5', 'away_team_goal_avg_last_5',
#        'home_team_goals_conceded_last_5', 'away_team_goals_conceded_last_5',
#        'home_team_form', 'away_team_form', 'home_team_home_win_rate',
#        'away_team_away_win_rate', 'home_team_home_goal_avg',
#        'away_team_away_goal_avg', 'head_to_head_win_home_team',
#        'head_to_head_draws', 'head_to_head_draws', 'head_to_head_goal_diff',
#        'head_to_head_goal_diff', 'is_weekend', 'month',
#        'days_since_last_match_home', 'match_importance', 'home_team_ranking',
#        'away_team_ranking', 'goal_avg_diff', 'win_rate_diff', 'ranking_diff',
#        'away_team_streak', 'away_team_last_match_result',
#        'away_team_points_last_3', 'away_team_score_diff_last_5',
#        'home_team_total_goals_scored_season',
#        'home_team_total_goals_conceded_season',
#        'home_team_matches_played_season', 'home_team_goal_diff_season',
#        'away_team_total_goals_scored_season',
#        'away_team_total_goals_conceded_season', 'home_team_encoder',
#        'away_team_encoder', 'match_day_of_week_encoder',
#        'season_phase_encoder']

In [10]:
list_feature_order = ["A", "B", "C"]

columns = ["A", "B", "c", "d"]

missing_columns = set(list_feature_order).difference(set(columns))

In [11]:
# FT_DIR = Path("datadase", "features", 'ft_df.csv')
df_prep = pd.read_csv(os.path.join(FT_DIR, "ft_df.csv"))

In [12]:
# df_validate = df_prep[df_prep["season"].isin([2025, 2024])].copy()
df_validate = df_prep[df_prep["season"].isin([2022])].copy()

In [13]:
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

is_float('11.1')  # True
# is_float('abc')   # False


True

In [14]:
for idx, row in df_validate.iterrows():
    home_team = row["home_team_encoder"]
    away_team = row["away_team_encoder"]
    pred, proba = model_trainer.predict_per_time(home_team=str(home_team), away_team=str(away_team))

    df_validate.loc[idx, "pred"] = pred
    df_validate.loc[idx, "proba_draw_%"] = round(proba[0], 5)*100
    df_validate.loc[idx, "proba_away_%"] = round(proba[1], 5)*100
    df_validate.loc[idx, "proba_home_%"] = round(proba[2], 5)*100

2025-04-15 15:52:11,549 | main | INFO | model_trainer | load_model | Modelo carregado com sucesso de: database\models\logistic_regression_model.pkl
2025-04-15 15:52:11,590 | main | INFO | model_trainer | load_model | Modelo carregado com sucesso de: database\models\logistic_regression_model.pkl
2025-04-15 15:52:11,619 | main | INFO | model_trainer | load_model | Modelo carregado com sucesso de: database\models\logistic_regression_model.pkl
2025-04-15 15:52:11,670 | main | INFO | model_trainer | load_model | Modelo carregado com sucesso de: database\models\logistic_regression_model.pkl
2025-04-15 15:52:11,705 | main | INFO | model_trainer | load_model | Modelo carregado com sucesso de: database\models\logistic_regression_model.pkl
2025-04-15 15:52:11,733 | main | INFO | model_trainer | load_model | Modelo carregado com sucesso de: database\models\logistic_regression_model.pkl
2025-04-15 15:52:11,783 | main | INFO | model_trainer | load_model | Modelo carregado com sucesso de: database\m

In [15]:
df_validate["pred"] = df_validate["pred"].astype(int)
df_validate["is_pred_ok"] = df_validate["pred"] == df_validate["winner"]

df_validate[["home_team_encoder","away_team_encoder","winner","pred", 'is_pred_ok', "proba_draw_%", "proba_away_%", "proba_home_%"]].head(20)

,home_team_encoder,away_team_encoder,winner,pred,is_pred_ok,proba_draw_%,proba_away_%,proba_home_%
3799,18,32,1,0,False,100.000,0.000,0.000
3800,2,17,1,0,False,68.363,30.372,1.264
3801,27,9,0,1,False,9.558,56.693,33.748
3802,12,20,2,0,False,53.218,34.367,12.415
3803,3,22,2,0,False,100.000,0.000,0.000
3804,6,11,0,0,True,100.000,0.000,0.000
3805,19,15,0,2,False,0.000,0.006,99.994
3806,4,0,2,0,False,99.999,0.001,0.000
3807,33,1,2,2,True,0.000,0.001,99.999
3808,24,7,1,2,False,0.000,0.035,99.965


In [16]:
# Como saber quais foram as variaveis decisivas que levaram o modelo a realizar a predição



In [17]:
end

NameError: name 'end' is not defined

In [18]:
df_validate["is_pred_ok"].value_counts(normalize=True).reset_index()

,is_pred_ok,proportion
0,False,0.597368
1,True,0.402632


In [ ]:
# Em um resultado aleatorio
df_validate["winner"].value_counts(normalize=True).reset_index()

,winner,proportion
0,2,0.442105
1,1,0.284211
2,0,0.273684


In [ ]:
end

NameError: name 'end' is not defined

----

In [ ]:
def get_soccer_data(country:str = "Brazil") -> pd.DataFrame | None:
    """
    Recupera os dados de futebol do banco de dados SQLite com base no país especificado.

    Parameters
    ----------
    country : str, optional
        Nome do país para filtrar os dados. O padrão é "Brazil".

    Returns
    -------
    pd.DataFrame | None
        Um DataFrame contendo os dados do banco de dados, ou None em caso de erro.
    """
    database_path = "database/soccer_data.db"
    conn = create_connection(database_path)

    if conn is None:
        logger.error("Falha ao conectar ao banco de dados. Encerrando a operação.")
        return None

    try:
        query = "SELECT * FROM soccer_data WHERE country = ?"
        df = pd.read_sql_query(query, conn, params=(country,))

        if df.empty:
            logger.warning(f"Nenhum dado encontrado para o país: {country}")
            return None

        logger.info(f"Dados carregados com sucesso para o país: {country}")

        # outra maneira
        # cursor = conn.cursor()
        # cursor.execute(query)
        # tables = cursor.fetchall()
        return df

    except sqlite3.Error as e:
        logger.error(f"Erro ao executar consulta: {e}")
        return None
    finally:
        conn.close()
        logger.debug("Conexão com o banco de dados fechada.")

In [ ]:
df = get_soccer_data()
df.head()

2025-04-12 21:01:10,930 | main | INFO | 2367544888 | get_soccer_data | Dados carregados com sucesso para o país: Brazil
2025-04-12 21:01:10,931 | main | DEBUG | 2367544888 | get_soccer_data | Conexão com o banco de dados fechada.


Conexão SQLite estabelecida.


,id,country,league,season,home_team,away_team,home_score,away_score,result,psch,pscd,psca,maxch,maxcd,maxca,avgch,avgcd,avgca,bfech,bfecd,datetime,hash,last_updated
0,5415,Brazil,Serie A,2012,Palmeiras,Portuguesa,1,1,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90,NaN,NaN,2012-05-19 22:30:00,10444097902145517897,2024-12-18 22:32:46
1,5416,Brazil,Serie A,2012,Sport Recife,Flamengo RJ,1,1,D,2.83,3.39,2.68,2.83,3.42,2.70,2.59,3.23,2.58,NaN,NaN,2012-05-19 22:30:00,7876314183501917566,2024-12-18 22:32:46
2,5417,Brazil,Serie A,2012,Figueirense,Nautico,2,1,H,1.60,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64,NaN,NaN,2012-05-20 01:00:00,9296066046964045682,2024-12-18 22:32:46
3,5418,Brazil,Serie A,2012,Botafogo RJ,Sao Paulo,4,2,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84,NaN,NaN,2012-05-20 20:00:00,3618841616446699339,2024-12-18 22:32:46
4,5419,Brazil,Serie A,2012,Corinthians,Fluminense,0,1,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89,NaN,NaN,2012-05-20 20:00:00,11994628649421207242,2024-12-18 22:32:46


In [ ]:
from modelagem.utils.feature.first_strategy import base_pre_processing, calculate_match_points

calculate_match_points(df)


,id,country,league,season,home_team,away_team,home_score,away_score,result,psch,pscd,psca,maxch,maxcd,maxca,avgch,avgcd,avgca,bfech,bfecd,datetime,hash,last_updated,winner,h_match_points,a_match_points
0,5415,Brazil,Serie A,2012,Palmeiras,Portuguesa,1,1,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90,NaN,NaN,2012-05-19 22:30:00,10444097902145517897,2024-12-18 22:32:46,0,1,1
1,5416,Brazil,Serie A,2012,Sport Recife,Flamengo RJ,1,1,D,2.83,3.39,2.68,2.83,3.42,2.70,2.59,3.23,2.58,NaN,NaN,2012-05-19 22:30:00,7876314183501917566,2024-12-18 22:32:46,0,1,1
2,5417,Brazil,Serie A,2012,Figueirense,Nautico,2,1,H,1.60,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64,NaN,NaN,2012-05-20 01:00:00,9296066046964045682,2024-12-18 22:32:46,2,3,0
3,5418,Brazil,Serie A,2012,Botafogo RJ,Sao Paulo,4,2,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84,NaN,NaN,2012-05-20 20:00:00,3618841616446699339,2024-12-18 22:32:46,2,3,0
4,5419,Brazil,Serie A,2012,Corinthians,Fluminense,0,1,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89,NaN,NaN,2012-05-20 20:00:00,11994628649421207242,2024-12-18 22:32:46,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4944,53084,Brazil,Serie A,2025,Flamengo RJ,Internacional,1,1,D,1.82,3.51,4.96,1.82,3.60,5.06,1.79,3.47,4.79,1.81,3.65,2025-03-30 00:00:00,14928226327939508637,2025-04-06 13:34:50,0,1,1
4945,53085,Brazil,Serie A,2025,Palmeiras,Botafogo RJ,0,0,D,1.70,3.59,5.89,1.70,3.70,6.30,1.67,3.60,5.70,1.70,3.90,2025-03-30 20:00:00,8115018700935595413,2025-04-06 13:34:50,0,1,1
4946,53086,Brazil,Serie A,2025,Vasco,Santos,2,1,H,2.33,3.13,3.51,2.41,3.20,3.51,2.33,3.07,3.34,2.36,3.30,2025-03-30 22:30:00,6586075529555824167,2025-04-06 13:34:50,2,3,0
4947,53087,Brazil,Serie A,2025,Bahia,Corinthians,1,1,D,1.80,3.56,5.04,1.90,3.70,5.04,1.82,3.48,4.61,1.86,3.65,2025-03-31 00:00:00,9743729930736042668,2025-04-06 13:34:50,0,1,1


In [ ]:
FT_DIR = Path("features")
output_path = os.path.join(FT_DIR, 'ft_df.csv')
df_prep = pd.read_csv(output_path)

In [ ]:
# ok, df_prep = base_pre_processing(df)

# if not ok:
#     print(df_prep)

In [ ]:
import json
import os
from pathlib import Path
import pandas as pd

# ==== CONFIGURAÇÕES ====
home_team = "Palmeiras"
away_team = "Corinthians"

# ==== CAMINHOS ====
BASE_DIR = Path("__file__").resolve().parent
MODEL_DIR = BASE_DIR.parent / "pred_soccer" / "database" / "models" / "logistic_regression_model.pkl"
FEATURES_PATH = BASE_DIR.parent / "pred_soccer" / "database" / "models" / "feature_order.json"
TEAM_MAPPING_PATH = BASE_DIR / "team_mapping.json"

# ==== CARREGAMENTO ====
model = model_trainer.load_model(str(MODEL_DIR))

with open(FEATURES_PATH, "r", encoding="utf-8") as f:
    feature_order = json.load(f)

with open(TEAM_MAPPING_PATH, "r", encoding="utf-8") as f:
    team_mapping = json.load(f)

# ==== MAPEAMENTO DE TIMES ====
dict_team_name_id = {k: v for k, v in team_mapping.items()}
dict_id_team_name = {v: k for k, v in team_mapping.items()}

# ==== COLUNAS PARA REMOVER ====
drop_columns_template = [
    "{type}_rank", "{type}_ls_rank", "{type}_days_ls_match", "{type}_points",
    "{type}_l_points", "{type}_l_wavg_points", "{type}_goals", "{type}_l_goals",
    "{type}_l_wavg_goals", "{type}_goals_sf", "{type}_l_goals_sf", "{type}_l_wavg_goals_sf",
    "{type}_wins", "{type}_draws", "{type}_losses", "{type}_win_streak",
    "{type}_loss_streak", "{type}_draw_streak", "winner"
]

drop_columns_at = [col.format(type="at") for col in drop_columns_template] + ["away_team_encoder"]
drop_columns_ht = [col.format(type="ht") for col in drop_columns_template] + ["home_team_encoder"]

# ==== SELECIONA E PREPARA OS DADOS ====
df_home = df_prep[df_prep["home_team_encoder"] == dict_team_name_id[home_team]].drop(columns=drop_columns_at).copy()
df_away = df_prep[df_prep["away_team_encoder"] == dict_team_name_id[away_team]].drop(columns=drop_columns_ht).copy()

# Usa a última linha disponível de cada time
df_merge = pd.merge(df_home.iloc[[-1]], df_away.iloc[[-1]])

# Remove colunas extras
df_merge.drop(columns="season", inplace=True, errors="ignore")

# Reordena as colunas conforme o modelo espera
df_merge = df_merge[feature_order]

# ==== PREDIÇÃO ====
prediction = model.predict(df_merge)[0]

# ==== INTERPRETAÇÃO ====
result_map = {
    0: "Empate",
    1: "Vitória do time visitante",
    2: "Vitória do time da casa"
}

print(f"Resultado previsto: {result_map.get(prediction, 'Desconhecido')}")


2025-04-12 21:01:11,119 | main | INFO | model_trainer | load_model | Modelo carregado com sucesso de: /media/guilherme/ssd_m2_data/py_new_projects/pred_soccer/database/models/logistic_regression_model.pkl


Resultado previsto: Vitória do time da casa


In [ ]:
BASE_DIR = os.path.dirname(Path("__file__").resolve().parent)
DATA_DIR = os.path.join(BASE_DIR, 'feature_eng', 'data', 'ft_df.csv')
MODEL_DIR = os.path.join(os.path.dirname(BASE_DIR), 'database', 'models')
LOG_DIR = os.path.join(os.path.dirname(BASE_DIR), 'logs')

# Carregar o modelo treinado
model_filename = os.path.join(MODEL_DIR, "logistic_regression_model.pkl")

# model = model_trainer.load_model(model_filename)
# logger.info("Modelo carregado com sucesso.")

# model py_new_projects/pred_soccer
# /media/guilherme/ssd_m2_data/py_new_projects/pred_soccer/database/models/logistic_regression_model.pkl
# /media/guilherme/ssd_m2_data/database/models/logistic_regression_model.pkl'
model_filename

'/media/guilherme/ssd_m2_data/database/models/logistic_regression_model.pkl'

In [ ]:
# utils.py

import json
import pandas as pddf_prep

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def get_team_mappings(team_mapping_json):
    return (
        {k: v for k, v in team_mapping_json.items()},
        {v: k for k, v in team_mapping_json.items()}
    )

def get_drop_columns():
    template = [
        "{type}_rank", "{type}_ls_rank", "{type}_days_ls_match", "{type}_points",
        "{type}_l_points", "{type}_l_wavg_points", "{type}_goals", "{type}_l_goals",
        "{type}_l_wavg_goals", "{type}_goals_sf", "{type}_l_goals_sf", "{type}_l_wavg_goals_sf",
        "{type}_wins", "{type}_draws", "{type}_losses", "{type}_win_streak",
        "{type}_loss_streak", "{type}_draw_streak", "winner"
    ]
    drop_at = [col.format(type="at") for col in template] + ["away_team_encoder"]
    drop_ht = [col.format(type="ht") for col in template] + ["home_team_encoder"]
    return drop_ht, drop_at

# predictor.py

import pandas as pd

# from utils import load_json, get_team_mappings, get_drop_columns

def prepare_input(df_prep, home_team, away_team, team_mapping, feature_order):
    dict_team_name_id, _ = get_team_mappings(team_mapping)
    drop_ht, drop_at = get_drop_columns()

    df_home = df_prep[df_prep["home_team_encoder"] == dict_team_name_id[home_team]].drop(columns=drop_at).copy()
    df_away = df_prep[df_prep["away_team_encoder"] == dict_team_name_id[away_team]].drop(columns=drop_ht).copy()

    df_merge = pd.merge(df_home.iloc[[-1]], df_away.iloc[[-1]])
    df_merge.drop(columns="season", inplace=True, errors="ignore")
    df_merge = df_merge[feature_order]

    return df_merge


import pandas as pd
from pathlib import Path

# ==== Times que deseja prever ====
HOME_TEAM = "Palmeiras"
AWAY_TEAM = "Corinthians"

# ==== Caminhos ====
BASE_DIR = Path("__file__").resolve().parent
MODEL_DIR = BASE_DIR.parent / "pred_soccer" / "database" / "models" / "logistic_regression_model.pkl"
FEATURES_PATH = BASE_DIR.parent / "pred_soccer" / "database" / "models" / "feature_order.json"
TEAM_MAPPING_PATH = BASE_DIR / "database" / "models" /  "team_mapping.json"


# Carrega modelo e dados auxiliares
model = model_trainer.load_model(str(MODEL_DIR))
feature_order = load_json(FEATURES_PATH)
team_mapping = load_json(TEAM_MAPPING_PATH)

df_prep.sort_values(by=["season"], ascending=True, inplace=True)
df_input = prepare_input(df_prep, HOME_TEAM, AWAY_TEAM, team_mapping, feature_order)
resultado = model_trainer.predict(model, df_input)

print(f"Resultado previsto: {resultado}")


2025-04-12 21:54:45,318 | main | INFO | model_trainer | load_model | Modelo carregado com sucesso de: /media/guilherme/ssd_m2_data/py_new_projects/pred_soccer/database/models/logistic_regression_model.pkl


Resultado previsto: [2]


In [ ]:
df_input

,home_team_encoder,away_team_encoder,ht_rank,ht_ls_rank,ht_days_ls_match,ht_points,ht_l_points,ht_l_wavg_points,ht_goals,ht_l_goals,ht_l_wavg_goals,ht_goals_sf,ht_l_goals_sf,ht_l_wavg_goals_sf,ht_wins,ht_draws,ht_losses,ht_win_streak,ht_loss_streak,ht_draw_streak,at_rank,at_ls_rank,at_days_ls_match,at_points,at_l_points,at_l_wavg_points,at_goals,at_l_goals,at_l_wavg_goals,at_goals_sf,at_l_goals_sf,at_l_wavg_goals_sf,at_wins,at_draws,at_losses,at_win_streak,at_loss_streak,at_draw_streak
0,27,11,13.0,2.0,-33.0,2.0,0.666667,0.75,3.0,0.333333,0.59375,9.0,1.0,1.0,0.0,2.0,4.0,0.0,0.0,2.0,9.0,7.0,-33.0,3.0,0.666667,0.4375,4.0,0.666667,0.648438,11.0,1.0,1.25,0.0,3.0,5.0,0.0,1.0,0.0


In [ ]:
model_trainer.predict(model=model, df_input=df_input)

array([2])

In [ ]:
df_prep[df_prep["away_team_encoder"]==11][['season', 'away_team_encoder', 'at_rank']].tail()

,season,away_team_encoder,at_rank
4658,2024,11,7.0
4650,2024,11,7.0
4683,2024,11,7.0
4604,2024,11,7.0
4947,2025,11,9.0


In [ ]:
FT_DIR = Path("features")
output_path = os.path.join(FT_DIR, 'ft_df.csv')
df_prep = pd.read_csv(output_path)

df_prep.shape

(4949, 40)

In [ ]:
output_path

'features/ft_df.csv'

In [ ]:
df_home

,season,home_team_encoder,ht_rank,ht_ls_rank,ht_days_ls_match,ht_points,ht_l_points,ht_l_wavg_points,ht_goals,ht_l_goals,ht_l_wavg_goals,ht_goals_sf,ht_l_goals_sf,ht_l_wavg_goals_sf,ht_wins,ht_draws,ht_losses,ht_win_streak,ht_loss_streak,ht_draw_streak
0,2012,27,18.0,-33.0,-33.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
31,2012,27,18.0,-33.0,3.062500,31.0,1.333333,0.722408,31.0,1.666667,1.269384,42.0,2.000000,2.184216,6.0,13.0,12.0,0.0,1.0,0.0
45,2012,27,18.0,-33.0,7.791667,39.0,1.000000,0.395685,38.0,1.000000,0.814763,59.0,2.000000,2.549605,8.0,15.0,20.0,0.0,2.0,0.0
68,2012,27,18.0,-33.0,7.104167,65.0,1.666667,1.302094,67.0,1.333333,0.893098,93.0,1.000000,0.731339,15.0,20.0,31.0,0.0,0.0,2.0
87,2012,27,18.0,-33.0,7.000000,90.0,2.333333,1.748131,95.0,1.666667,1.053282,128.0,0.666667,0.634156,22.0,24.0,41.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4859,2024,27,2.0,1.0,5.854167,320.0,1.000000,0.405631,319.0,1.666667,0.654778,401.0,1.666667,1.254065,81.0,77.0,140.0,0.0,2.0,0.0
4879,2024,27,2.0,1.0,4.062500,346.0,1.000000,0.488321,345.0,0.333333,0.235976,434.0,1.000000,1.441099,88.0,82.0,150.0,0.0,2.0,0.0
4909,2024,27,2.0,1.0,3.083333,380.0,0.666667,0.848961,371.0,0.333333,0.266566,468.0,1.333333,0.701467,95.0,95.0,159.0,0.0,0.0,2.0
4938,2024,27,2.0,1.0,3.770833,409.0,2.000000,1.270317,401.0,1.000000,0.793889,501.0,0.666667,1.000265,103.0,100.0,167.0,0.0,1.0,0.0


In [ ]:
end

NameError: name 'end' is not defined

In [ ]:

# Tarefa: 
# Passo 1: Criar um arquivo CSV com os dados de cada time
    # Ter um arquivo contendo os dados de cada time
        # Nome do time A e Nome do time B
        # Data do jogo
# Passo 2:
    # Realizar predição do resultado do jogo
# Passo 3:
    # Salvar resultados e mortar para usuario



In [ ]:



df_prep.corr()

,season,home_team_encoder,away_team_encoder,winner,ht_rank,ht_ls_rank,ht_days_ls_match,ht_points,ht_l_points,ht_l_wavg_points,ht_goals,ht_l_goals,ht_l_wavg_goals,ht_goals_sf,ht_l_goals_sf,ht_l_wavg_goals_sf,ht_wins,ht_draws,ht_losses,ht_win_streak,ht_loss_streak,ht_draw_streak,at_rank,at_ls_rank,at_days_ls_match,at_points,at_l_points,at_l_wavg_points,at_goals,at_l_goals,at_l_wavg_goals,at_goals_sf,at_l_goals_sf,at_l_wavg_goals_sf,at_wins,at_draws,at_losses,at_win_streak,at_loss_streak,at_draw_streak
season,1.000000,-0.122937,-0.123189,-0.024426,-0.002535,0.219607,0.005446,0.043544,0.066162,0.053020,0.032639,0.063751,0.034514,-0.034524,-0.052012,-0.056651,0.048547,0.027476,-0.052856,0.029158,-0.055754,-0.002226,0.002423,0.215903,0.005916,0.043548,0.059310,0.048676,0.032609,0.057155,0.029402,-0.034494,-0.047131,-0.052841,0.048553,0.027476,-0.052853,0.028461,-0.052256,-0.002226
home_team_encoder,-0.122937,1.000000,-0.028631,0.017439,-0.045768,0.000762,-0.006750,-0.007406,0.003010,0.004693,-0.001187,-0.005639,-0.001191,0.003103,-0.006797,-0.010148,-0.006735,-0.009172,0.005726,0.018813,0.010911,-0.004636,0.003385,-0.032208,0.009494,-0.011586,0.005834,0.006614,-0.003288,-0.001074,0.003090,0.004614,-0.010633,-0.013710,-0.012288,-0.009172,0.008864,0.021200,0.008176,-0.004636
away_team_encoder,-0.123189,-0.028631,1.000000,-0.005890,0.002004,-0.031247,-0.008484,-0.008159,-0.022291,-0.030594,-0.000090,-0.009084,-0.000040,0.008739,0.021727,0.028806,-0.008878,-0.005792,0.012239,-0.025348,0.022876,0.016604,-0.044849,-0.000437,-0.002693,-0.003894,-0.017184,-0.023682,0.002749,-0.004963,0.004850,0.006697,0.018430,0.024880,-0.003213,-0.005792,0.009036,-0.018957,0.018060,0.016604
winner,-0.024426,0.017439,-0.005890,1.000000,-0.191656,0.030158,-0.022263,0.017222,0.001992,0.003700,0.015901,-0.005497,-0.008313,0.011179,-0.005962,-0.009850,0.016703,0.018254,0.012104,0.003272,-0.009026,0.021626,0.155345,-0.027974,-0.017953,0.003940,0.003166,0.004115,0.006240,-0.003547,-0.005912,0.018127,-0.007596,-0.011820,-0.000941,0.018254,0.022081,0.001197,-0.003547,0.021626
ht_rank,-0.002535,-0.045768,0.002004,-0.191656,1.000000,-0.163351,0.008299,-0.050658,-0.004139,-0.001330,-0.033415,-0.003875,-0.000900,0.027456,0.006171,-0.000498,-0.067712,0.001258,0.042144,0.013939,0.024072,-0.014465,-0.054020,0.008636,0.008810,0.003167,0.004443,0.005776,0.002577,0.001352,0.002497,0.001567,0.001832,-0.003297,0.003782,0.001258,0.001721,0.018784,0.020264,-0.014465
ht_ls_rank,0.219607,0.000762,-0.031247,0.030158,-0.163351,1.000000,0.011562,0.012903,0.013428,0.025765,0.000134,-0.000958,0.001488,-0.019549,-0.017529,-0.023666,0.016939,0.000590,-0.012778,0.030830,-0.026457,0.013099,0.009571,0.182935,0.011785,-0.001326,0.011927,0.025084,-0.010270,-0.003786,-0.000643,-0.012066,-0.015233,-0.022006,-0.001961,0.000590,-0.002093,0.032861,-0.024824,0.013099
ht_days_ls_match,0.005446,-0.006750,-0.008484,-0.022263,0.008299,0.011562,1.000000,0.210852,0.059970,0.057743,0.215059,0.051456,0.044959,0.218477,0.023261,0.015268,0.207365,0.214991,0.218574,0.035010,0.012083,0.008542,-0.006498,0.010287,0.947316,0.211653,0.058754,0.057189,0.215755,0.051881,0.045170,0.217989,0.022695,0.014898,0.208429,0.214991,0.218013,0.033061,0.013361,0.008542
ht_points,0.043544,-0.007406,-0.008159,0.017222,-0.050658,0.012903,0.210852,1.000000,0.042298,0.045256,0.991402,0.016178,0.023410,0.966463,0.031858,0.034739,0.997517,0.978075,0.940165,0.039026,-0.002209,-0.011284,0.003427,0.002898,0.215414,0.993251,0.044132,0.045153,0.987161,0.019781,0.025641,0.969572,0.028887,0.032897,0.988553,0.978075,0.945425,0.042935,0.001072,-0.011284
ht_l_points,0.066162,0.003010,-0.022291,0.001992,-0.004139,0.013428,0.059970,0.042298,1.000000,0.874330,0.034585,0.631884,0.552740,0.009342,-0.563253,-0.486887,0.050116,0.017938,-0.002199,0.595558,-0.567105,0.008286,-0.004136,-0.003386,0.058412,0.041268,0.954000,0.842416,0.033940,0.599555,0.528864,0.009810,-0.542904,-0.472143,0.048750,0.017938,-0.001418,0.578872,-0.546130,0.008286
ht_l_wavg_po

In [ ]:
# Ideia: gerar uma base de dados onde seja possivel 

# Qual a melhor estrutura 